# Kukavica

Kukavica bo izlegla $16$ jajc in jih podtaknila v $12$ gnezd, ki pripadajo dvema taščicama, štirim vrtnim penicam, trem travniškim cipam, dvema belima pastiricama in eni sovi. V vsako gnezdo lahko izleže največ tri jajca, pri čemer je verjetnost, da mladiči v gnezdu $i$ preživijo, enaka $p_{ij}$, kjer je $j$ število podtaknjenih jajc v gnezdu $i$ (preživijo bodisi vsi ali noben mladič v posameznem gnezdu). Pri vsaki od petih vrst ptic želi izleči vsaj eno jajce, pri taščicah pa želi izleči strogo več jajc kot pri belih pastiricah. Poleg tega pri drugi beli pastirici ne bo odložila jajca, če bo pri prvi taščici odložila dve jajci ali več. Kukavica želi maksimizirati pričakovano število preživelih mladičev.

## Parametri

Pripravimo matriko (seznam seznamov) z verjetnostmi preživetja mladičev v odvisnosti od števila podtaknjenih jajc v gnezdu - velja torej `pr[i][j] =` $p_{ij}$ ($0 \le i \le 11$, $0 \le j \le 3$).

In [1]:
pr = [
    [1, 0.8, 0.7, 0.4], # 0:  taščica 1
    [1, 0.8, 0.6, 0.4], # 1:  taščica 2
    [1, 0.9, 0.8, 0.5], # 2:  vrtna penica 1
    [1, 0.8, 0.8, 0.6], # 3:  vrtna penica 2
    [1, 0.8, 0.8, 0.5], # 4:  vrtna penica 3
    [1, 0.9, 0.7, 0.6], # 5:  vrtna penica 4
    [1, 0.8, 0.7, 0.4], # 6:  travniška cipa 1
    [1, 0.8, 0.8, 0.4], # 7:  travniška cipa 2
    [1, 0.7, 0.7, 0.5], # 8:  travniška cipa 3
    [1, 0.7, 0.5, 0.4], # 9:  bela pastirica 1
    [1, 0.7, 0.7, 0.4], # 10: bela pastirica 2
    [1, 0.6, 0.4, 0.1]  # 11: sova
]

In [2]:
n = len(pr)    # število gnezd
m = len(pr[0]) # maksimalno število jajc v gnezdu

## Celoštevilski linearni program

Zapisali bomo sledeči celoštevilski linearni program:

\begin{align*}
\max \sum_{i=0}^{11} \sum_{j=1}^3 j \ p_{ij} \ x_{ij} & \\
\text{p. p.} \quad
\forall i \in \{0, 1, \dots, 11\} \ \forall j \in \{1, 2, 3\}. \ 0 \le x_{ij} &\le 1, \ x_{ij} \in \mathbb{Z} \\
\forall i \in \{0, 1, \dots, 11\}. \sum_{j=1}^3 x_{ij} &\le 1 \\
\sum_{i=0}^{11} \sum_{j=1}^3 j \ x_{ij} &= 16 \\
\sum_{i=0}^1 \sum_{j=1}^3 x_{ij} &\ge 1 \\
\sum_{i=2}^5 \sum_{j=1}^3 x_{ij} &\ge 1 \\
\sum_{i=6}^8 \sum_{j=1}^3 x_{ij} &\ge 1 \\
\sum_{i=9}^{10} \sum_{j=1}^3 x_{ij} &\ge 1 \\
\sum_{j=1}^3 x_{11,j} &\ge 1 \\
\sum_{i=0}^1 \sum_{j=1}^3 j \ x_{ij} - \sum_{i=9}^{10} \sum_{j=1}^3 j \ x_{ij} &\ge 1 \\
\sum_{j=2}^3 x_{0j} + \sum_{j=1}^3 x_{10,j} &\le 1 \\
\end{align*}

In [3]:
p = MixedIntegerLinearProgram(maximization=True)
x = p.new_variable(binary=True)

In [4]:
p.set_objective(sum(j * pr[i][j] * x[i, j] for j in range(1, m) for i in range(n)))
p.add_constraint(sum(j * x[i, j] for j in range(1, m) for i in range(n)) == 16)
p.add_constraint(sum(x[i, j] for j in range(1, m) for i in [0, 1]) >= 1)
p.add_constraint(sum(x[i, j] for j in range(1, m) for i in [2, 3, 4, 5]) >= 1)
p.add_constraint(sum(x[i, j] for j in range(1, m) for i in [6, 7, 8]) >= 1)
p.add_constraint(sum(x[i, j] for j in range(1, m) for i in [9, 10]) >= 1)
p.add_constraint(sum(x[11, j] for j in range(1, m)) >= 1)
p.add_constraint(sum(j * x[i, j] for j in range(1, m) for i in [0, 1]) >=
                 sum(x[i, j] for j in range(1, m) for i in [9, 10]) + 1)
p.add_constraint(x[0, 2] + x[0, 3] + x[10, 1] + x[10, 2] + x[10, 3] <= 1)

for i in range(n):
    p.add_constraint(sum(x[i, j] for j in range(1, m)) <= 1)

Z metodo `solve` dobimo optimalno vrednost ciljne funkcije.

In [5]:
p.solve()

12.399999999999999

Z metodo `get_values` dobimo vrednosti spremenljivk pri najdeni optimalni rešitvi.

In [6]:
res = p.get_values(x)
res

{(0, 1): 1,
 (1, 1): 1,
 (2, 1): 0,
 (3, 1): 0,
 (4, 1): 0,
 (5, 1): 1,
 (6, 1): 1,
 (7, 1): 0,
 (8, 1): 0,
 (9, 1): 1,
 (10, 1): 0,
 (11, 1): 1,
 (0, 2): 0,
 (1, 2): 0,
 (2, 2): 1,
 (3, 2): 1,
 (4, 2): 1,
 (5, 2): 0,
 (6, 2): 0,
 (7, 2): 1,
 (8, 2): 1,
 (9, 2): 0,
 (10, 2): 0,
 (11, 2): 0,
 (0, 3): 0,
 (1, 3): 0,
 (2, 3): 0,
 (3, 3): 0,
 (4, 3): 0,
 (5, 3): 0,
 (6, 3): 0,
 (7, 3): 0,
 (8, 3): 0,
 (9, 3): 0,
 (10, 3): 0,
 (11, 3): 0}

Izpišimo, v katera gnezda naj kukavica odloži jajca.

In [7]:
kuk = {i: j for (i, j), v in res.items() if v == 1}
kuk

{0: 1, 1: 1, 5: 1, 6: 1, 9: 1, 11: 1, 2: 2, 3: 2, 4: 2, 7: 2, 8: 2}